# Семантическое ядро с интеграцией сервера OpenBnB MCP

В этой тетради показано, как использовать Семантическое ядро с реальным сервером OpenBnB MCP для поиска настоящих предложений Airbnb с использованием MCPStdioPlugin. Для доступа к LLM используется Microsoft Foundry. Чтобы настроить переменные окружения, вы можете следовать уроку [Setup Lesson](/00-course-setup/README.md)


## Импорт необходимых пакетов


In [ ]:
# Import cell - Updated imports
import json
import os
import asyncio
import subprocess
import sys


from dotenv import load_dotenv
from IPython.display import display, HTML
from typing import Annotated

from semantic_kernel.agents import ChatCompletionAgent, ChatHistoryAgentThread
from semantic_kernel.connectors.ai.open_ai import AzureChatCompletion
from semantic_kernel.connectors.mcp import MCPStdioPlugin
from semantic_kernel.contents import FunctionCallContent, FunctionResultContent, StreamingTextContent

## Создание подключения плагина MCP

Мы подключимся к [серверу OpenBnB MCP](https://github.com/openbnb-org/mcp-server-airbnb) с помощью MCPStdioPlugin. Этот сервер предоставляет функциональность поиска Airbnb через пакет @openbnb/mcp-server-airbnb.


## Создание клиента

В этом примере мы будем использовать Microsoft Foundry для доступа к нашему LLM. Убедитесь, что ваши переменные окружения настроены правильно.


## Конфигурация окружения

Настройте параметры Azure OpenAI. Убедитесь, что у вас установлены следующие переменные окружения:
- `AZURE_OPENAI_CHAT_DEPLOYMENT_NAME`
- `AZURE_OPENAI_ENDPOINT`
- `AZURE_OPENAI_API_KEY`


In [ ]:
# Creating the Client cell - Updated for Azure
load_dotenv()

# Azure OpenAI configuration
# Ensure these environment variables are set:
# - AZURE_OPENAI_CHAT_DEPLOYMENT_NAME
# - AZURE_OPENAI_ENDPOINT
# - AZURE_OPENAI_API_KEY (optional if using DefaultAzureCredential)

chat_completion_service = AzureChatCompletion(
    deployment_name=os.getenv("AZURE_OPENAI_CHAT_DEPLOYMENT_NAME"),
    endpoint=os.getenv("AZURE_OPENAI_ENDPOINT"),
    # Optional - will use DefaultAzureCredential if not set
    api_key=os.getenv("AZURE_OPENAI_API_KEY"),
)

## Понимание интеграции OpenBnB MCP

Этот блокнот подключается к **реальному серверу OpenBnB MCP**, который предоставляет настоящую функциональность поиска Airbnb.

### Как это работает:

1. **MCPStdioPlugin**: Использует стандартное взаимодействие ввода/вывода с сервером MCP
2. **Реальный пакет NPM**: Загружает и запускает `@openbnb/mcp-server-airbnb` через npx
3. **Живые данные**: Возвращает реальные данные об объектах Airbnb через их API
4. **Обнаружение функций**: Агент автоматически находит доступные функции сервера MCP

### Доступные функции:

Сервер OpenBnB MCP обычно предоставляет:
- **search_listings** - Поиск объектов Airbnb по локации и критериям
- **get_listing_details** - Получение подробной информации о конкретных объектах
- **check_availability** - Проверка доступности на конкретные даты
- **get_reviews** - Получение отзывов об объектах
- **get_host_info** - Получение информации о хозяевах объектов

### Требования:

- **Node.js** установлен на вашей системе
- **Подключение к Интернету** для загрузки пакета сервера MCP
- **NPX** доступен (входит в комплект Node.js)

### Тестирование подключения:

Вы можете вручную протестировать сервер MCP, запустив:
```bash
npx -y @openbnb/mcp-server-airbnb
```

Это скачает и запустит сервер OpenBnB MCP, к которому затем Semantic Kernel подключится для получения реальных данных Airbnb.


## Запуск агента с сервером OpenBnB MCP

Теперь мы запустим ИИ-агента, который подключается к серверу OpenBnB MCP для поиска реального жилья Airbnb в Стокгольме для 2 взрослых и 1 ребенка. Не стесняйтесь изменить список `user_inputs`, чтобы изменить критерии поиска.


In [ ]:
user_inputs = [
    "Find Airbnb in Stockholm for 2 adults 1 kid",
]


async def main():
    """Main function to run the MCP-enabled agent with real OpenBnB server using Azure OpenAI"""

    try:
        print("🚀 Starting with Azure OpenAI...")
        
        # Verify environment variables
        print("🔍 Checking Azure environment variables...")
        required_vars = ["AZURE_OPENAI_CHAT_DEPLOYMENT_NAME", "AZURE_OPENAI_ENDPOINT", "AZURE_OPENAI_API_KEY"]
        for var in required_vars:
            if os.getenv(var):
                print(f"✅ {var} is set")
            else:
                print(f"❌ {var} is NOT set")
        
        print("\n🔧 Creating MCP Plugin...")
        
        # Create MCP plugin connection to real OpenBnB server
        # Based on the GitHub repo, the server doesn't need special env vars
        async with MCPStdioPlugin(
            name="AirbnbSearch",
            description="Search for Airbnb accommodations using OpenBnB MCP server",
            command="npx",
            args=["-y", "@openbnb/mcp-server-airbnb"],
        ) as airbnb_plugin:

            print("✅ MCP Plugin created and connected")
            
            # Wait a moment for the server to fully initialize
            await asyncio.sleep(2)
            
            # Try to list available tools
            try:
                tools = await airbnb_plugin.get_tools()
                print(f"🔧 Available tools: {[tool.name for tool in tools]}")
            except Exception as e:
                print(f"⚠️ Could not list tools: {str(e)}")

            # Create the Azure OpenAI service with proper configuration
            print("\n🤖 Creating Azure OpenAI service...")
            service = AzureChatCompletion(
                deployment_name=os.getenv("AZURE_OPENAI_CHAT_DEPLOYMENT_NAME"),
                endpoint=os.getenv("AZURE_OPENAI_ENDPOINT"),
                api_key=os.getenv("AZURE_OPENAI_API_KEY"),
            )
            
            # Create agent with the service instance
            agent = ChatCompletionAgent(
                service=service,
                name="AirbnbAgent",
                instructions="""You are an Airbnb search assistant. Use the available functions to search for properties. 
                Format results in a clear HTML table with columns for property name, price, rating, and link.""",
                plugins=[airbnb_plugin],
            )

            print("✅ Agent created with Azure OpenAI")

            # Process each user input
            thread: ChatHistoryAgentThread | None = None

            for user_input in user_inputs:
                print(f"\n🔍 User: {user_input}")
                
                try:
                    # Use the simpler get_response method
                    response = await agent.get_response(messages=user_input, thread=thread)
                    thread = response.thread
                    
                    # Process the response text
                    response_text = str(response)
                    
                    # Remove any markdown code blocks around HTML
                    response_text = response_text.replace('```html', '').replace('```', '')
                    
                    # Display the result
                    print(f"🤖 {response.name}: {response_text[:200]}..." if len(response_text) > 200 else response_text)
                    
                    # If response contains HTML table, display it properly
                    if '<table' in response_text.lower():
                        # Add CSS styling for better table rendering
                        table_css = """
                        <style>
                            .airbnb-results table {
                                border-collapse: collapse;
                                width: 100%;
                                margin: 10px 0;
                            }
                            .airbnb-results th, .airbnb-results td {
                                border: 1px solid #ddd;
                                padding: 8px;
                                text-align: left;
                            }
                            .airbnb-results th {
                                background-color: #f2f2f2;
                                font-weight: bold;
                            }
                            .airbnb-results tr:nth-child(even) {
                                background-color: #f9f9f9;
                            }
                            .airbnb-results a {
                                color: #1976d2;
                                text-decoration: none;
                            }
                            .airbnb-results a:hover {
                                text-decoration: underline;
                            }
                        </style>
                        """
                        html_output = f'{table_css}<div class="airbnb-results">{response_text}</div>'
                        display(HTML(html_output))
                    else:
                        # Display as regular text if no table
                        display(HTML(f'<div class="airbnb-results">{response_text}</div>'))
                        
                except Exception as e:
                    print(f"❌ Error processing user input: {str(e)}")
                    import traceback
                    traceback.print_exc()
                
            # Cleanup
            if thread:
                await thread.delete()
                print("🧹 Thread cleaned up")
                
    except Exception as e:
        print(f"❌ Main error: {str(e)}")
        import traceback
        traceback.print_exc()

# Run the main function
print("🚀 Starting MCP Agent...")
await main()
print("✅ Done!")

# Резюме
Поздравляем! Вы успешно создали AI-агента, который интегрируется с поиском жилья в реальном мире с использованием протокола Model Context Protocol (MCP):

## Использованные технологии:
- Semantic Kernel — для создания интеллектуальных агентов с Azure OpenAI
- Microsoft Foundry — для возможностей LLM и чатов с автозаполнением
- MCP (Model Context Protocol) — для стандартизированной интеграции инструментов
- OpenBnB MCP Server — для реального поиска жилья на Airbnb
- Node.js/NPX — для запуска внешнего MCP сервера

## Чему вы научились:
- Интеграция MCP: подключение агентов Semantic Kernel к внешним MCP серверам
- Доступ к данным в реальном времени: поиск реальных объектов Airbnb через живые API
- Протокольное взаимодействие: использование общения через stdio между агентом и MCP сервером
- Обнаружение функций: автоматическое определение доступных функций с MCP серверов
- Потоковые ответы: захват и логирование вызовов функций в реальном времени
- Отрисовка HTML: форматирование ответов агента с помощью стилизованных таблиц и интерактивных элементов

## Следующие шаги:
- Интеграция дополнительных MCP серверов (погода, рейсы, рестораны)
- Создание мультиагентной системы с использованием протоколов MCP и A2A
- Создание собственных MCP серверов для ваших источников данных
- Реализация постоянной памяти разговора между сессиями
- Развертывание агента на Azure Functions с оркестрацией MCP сервера
- Добавление аутентификации пользователей и возможностей бронирования


---

<!-- CO-OP TRANSLATOR DISCLAIMER START -->
**Отказ от ответственности**:  
Данный документ был переведен с использованием сервиса автоматического перевода [Co-op Translator](https://github.com/Azure/co-op-translator). Хотя мы стремимся к точности, просим учитывать, что автоматический перевод может содержать ошибки или неточности. Оригинальный документ на его исходном языке следует считать авторитетным источником. Для получения критически важной информации рекомендуется обращаться к профессиональному переводу, выполненному человеком. Мы не несем ответственности за любые недоразумения или неправильные толкования, возникшие в результате использования данного перевода.
<!-- CO-OP TRANSLATOR DISCLAIMER END -->
